In [124]:
# Newton's forward, backward and divided difference

import math
import numpy as np

class Interpolation:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.forward_diff_row = []
        self.backward_diff_row = []
        self.divided_diff = []
        self.gauss_forward_diff = []
        self.gauss_backward_diff = []
        self.difference_table()
        self.u = symbols('u')   # for polynomial in u
        self.var = symbols('x') # real variable for polynomial

    def difference_table(self):
        # self.forward_diff_row = []
        # self.backward_diff_row = []
        curr_col = self.y.copy()
        curr_len = len(curr_col)
        
        while(curr_len > 1):
            for i in range (0, curr_len - 1):
                sub = curr_col[i + 1] - curr_col[i]
                if i == 0:
                    self.forward_diff_row.append(sub)
                if i == (curr_len - 2):
                    self.backward_diff_row.append(sub)
                curr_col.append(sub)
            curr_col = curr_col[curr_len:]
            curr_len = len(curr_col)  

    def calculating_u(self, u, n, is_forward):
        temp = 1
        if is_forward:
            for i in range(0, n):
                temp *= (u - i)
        else:
            for i in range(0, n):
                temp *= (u + i)
        return temp

    def newtons_forward_difference(self, x_point):
        h = self.x[1] - self.x[0]
        u = (x_point - self.x[0]) / h
        # self.difference_table()
        n = len(self.forward_diff_row)
        yx = self.y[0]
        for i in range(1, n + 1):  # +1 ensures the last difference (delta y0) is included
            yx = yx + (self.calculating_u(u, i, True) * self.forward_diff_row[i-1] / math.factorial(i))
        return yx

    def newtons_forward_difference_polynomial(self):
        h = self.x[1] - self.x[0]
        u = (self.var - self.x[0]) / h  # Takes into account 'x' as a variable
        poly_expression = self.y[0]
        for i in range(1, len(self.forward_diff_row) + 1):
            term = 1
            for j in range(i):
                term *= (u - j)
            poly_expression += (term * self.forward_diff_row[i - 1]) / factorial(i)
        return simplify(poly_expr.expand())

    def newtons_backward_difference(self, x_point):
        h = self.x[1] - self.x[0]
        u = (x_point - self.x[len(self.x) - 1]) / h
        # self.difference_table()
        n = len(self.backward_diff_row)
        yx = self.y[len(self.y) - 1]
        for i in range(1, n + 1):  
            yx += (self.calculating_u(u, i, False) * self.backward_diff_row[i-1] / math.factorial(i))
        return yx

    def newtons_backward_difference_polynomial(self):
        h = self.x[1] - self.x[0]
        u = (self.var - self.x[-1]) / h
        poly_expression = self.y[-1]
        for i in range(1, len(self.backward_diff_row) + 1):
            term = 1
            for j in range(i):
                term *= (u + j)
            poly_expr += (term * self.backward_diff_row[i - 1]) / factorial(i)
        return simplify(poly_expr.expand())

    def calculating_divided_x(self, x_point, n):
        temp = 1
        for i in range(0, n):
            temp *= (x_point - self.x[i])
        return temp

    def newtons_divided_difference(self, x_point):
        n = len(self.y)
        self.divided_diff = []
        curr_col = self.y.copy()
        curr_x = self.x.copy()
        
        for i in range(1, n):
            for j in range(n - i):
                curr_col[j] = (curr_col[j + 1] - curr_col[j]) / (curr_x[i + j] - curr_x[j])
            self.divided_diff.append(curr_col[0])
        fx = self.y[0]
        for i in range(len(self.divided_diff)):
            fx += self.calculating_divided_x(x_point, i + 1) * self.divided_diff[i]
        return fx

    def newtons_divided_difference_polynomial(self):
        if not self.divided_diff:
            self.divided_difference_table()
        poly_expr = self.divided_diff[0]
        term = 1
        for i in range(1, len(self.x)):
            term *= (self.var - self.x[i - 1])
            poly_expr += self.divided_diff[i] * term
        return simplify(poly_expr.expand())

    def derivative_at_point(self, method="divided", x_val=0):
        if method == "forward":
            poly = self.newtons_forward_difference_polynomial()
        elif method == "backward":
            poly = self.newtons_backward_difference_polynomial()
        else:
            poly = self.newtons_divided_difference_polynomial()

        derivative = diff(poly, self.var)
        f_prime = lambdify(self.var, derivative, "numpy")
        return float(f_prime(x_val)), poly, derivative

        # Converting y to 2D list for having access to the whole table
        # curr_col = [[0 for i in range(n)]
        #                for j in range(n)];
        # for i in range(0, n):
        #     curr_col[i][0] = self.y[i]
        # print(curr_col)

        # Inside j for loop
                # curr_col[j][i] = (curr_col[j + 1][i - 1] - curr_col[j][i - 1]) / (curr_x[i + j] - curr_x[j])
                # curr_col[j + 1][i - 1] - curr_col[j][i - 1] -> going along the column and subtracting next value in row from current value.
                # Column remains constant as (i - 1) because indexing starts from 0

        # for i in range(n):
        #     for j in range(n - i):
        #         print(curr_col[i], end = "\t")
        #     print("\n")

    def Lagrange_Interpolation(self, x_point):
        ratio = 1.0
        result = 0.0

        for i in range(len(self.x)):
            ratio = 1.0
            for j in range(len(self.x)):
                if i != j:
                    ratio *= (x_point - self.x[j]) / (self.x[i] - self.x[j])
            result += self.y[i] * ratio
        return result

    def stirling_coeff(self, p, i):
        coeff = 1.0
        if i%2 != 0:
            coeff *= p
            for k in range(1, (i + 1)//2):
                coeff *= ((p**2) - (k**2))
        else:
            coeff *= (p**2)
            for k in range(1, i//2):
                coeff *= ((p**2) - (k**2))
        return coeff / math.factorial(i)

    def Stirlings_central_difference(self, x_point):
        n = len(self.y)
        self.gauss_forward_diff = []
        self.gauss_backward_diff = []
        curr_col = self.y.copy()
        curr_x = self.x.copy()
        midpoint_forward = midpoint_backward = n // 2
        
        for i in range(1, n):
            new_col = []
            for j in range(n - i):
                new_col.append(curr_col[j + 1] - curr_col[j])
            curr_col = new_col

            length = len(curr_col)
            f_index = b_index = 0
            if i % 2 != 0:  
                b_index = midpoint_backward - (i // 2) - 1  # Go upwards
            else:           
                b_index = midpoint_backward - (i // 2)  # Stay on the same index

            # Backward and forward has common data at even intervals of delta_y
            f_index = midpoint_forward - (i // 2) # When index is odd, we stay on the same row, when even, move one step upwards
    
            # Checking whether indices are within bounds
            if 0 <= f_index < length:
                self.gauss_forward_diff.append(round(curr_col[f_index], 3))
            if 0 <= b_index < length:
                self.gauss_backward_diff.append(round(curr_col[b_index], 3))

        h = self.x[1] - self.x[0]
        p = (x_point - self.x[n // 2]) / h
        yx = self.y[n // 2]
        for i in range(1, len(self.gauss_forward_diff) + 1):
            # Averaging the odd terms
            diff = 0
            if i%2 != 0:
                diff = (self.gauss_forward_diff[i - 1] + self.gauss_backward_diff[i - 1]) / 2
            else:
                diff = self.gauss_forward_diff[i - 1]
                        
            coeff = self.stirling_coeff(p, i)
            yx += coeff * diff
        return yx

                
        # n = len(self.y)
        # self.gauss_forward_diff = []
        # # Converting y to 2D list for having access to the whole table
        # curr_col = [[0 for i in range(n)]
        #                for j in range(n)];
        # for i in range(0, n):
        #     curr_col[i][0] = self.y[i]
        # midpoint = math.floor(len(curr_col) / 2)
        # self.gauss_forward_diff.append(curr_col[midpoint][0])

        # for i in range(1, n):            
        #     for j in range(n - i):
        #         curr_col[j][i] = (curr_col[j + 1][i - 1] - curr_col[j][i - 1])
            # self.gauss_forward_diff.append(curr_col[midpoint])
            # if i%2 != 0:
            #     midpoint -= 1    

In [126]:
# i1 = Interpolation([1891, 1901, 1911, 1921, 1931], [46, 66, 81, 93, 101])
# i1 = Interpolation([0.0, 0.1, 0.2, 0.3, 0.4], [1.0, 0.9975, 0.99, 0.9776, 0.8604])

# result1 = i1.newtons_forward_difference(1896)
# poly = i1.newtons_forward_difference_polynomial()
# dy_dx, poly_expression, derivative_expression = i1.derivative_at_point(method = "forward", x_val = 0)
# print(dy_dx)
# print(i1.forward_diff_row)
# i1.newtons_forward_difference_polynomial()

# result2 = i1.newtons_backward_difference(1930)
# print(result2)

0.250333333333336


In [ ]:
i1 = Interpolation([0.0, 0.1, 0.2, 0.3, 0.4], [1.0, 0.9975, 0.99, 0.9776, 0.8604])

# result1 = i1.newtons_forward_difference(1896)
poly = i1.newtons_forward_difference_polynomial()
dy_dx, poly_expression, derivative_expression = i1.derivative_at_point(method = "forward", x_val = 0)
print(dy_dx)

In [66]:
# i4 = Interpolation([0,4,8,12,16], [14,24,32,35,40])
# i4.Stirlings_central_difference(9)
# print(i4.gauss_forward_diff)

# i5 = Interpolation([1901, 1911, 1921, 1931, 1941, 1951], [12, 15, 20, 27, 39, 52])
# print(i5.Stirlings_central_difference(1936))
# print(i5.gauss_backward_diff)

i6 = Interpolation([2,6,10,14,18], [21.857, 21.025, 20.132, 19.145, 18.057])
print(i6.Stirlings_central_difference(11))
# print(i6.gauss_forward_diff)
# print(i6.gauss_backward_diff)

19.8947802734375


In [67]:
# ni = Newtons_Interpolation([0,1,2,3], [1,0,1,10])
# result = ni.forward_difference(-1)
# print(result)

In [113]:
i2 = Interpolation([4,5,7,10,11,13], [48,100,294,900,1210,2028])

print(i2.newtons_divided_difference(8))
print(i2.divided_diff)

448.0
[52.0, 15.0, 1.0, 0.0, 0.0]


In [69]:
i3 = Interpolation([5,6,9,11], [12,13,14,16])
print(i3.Lagrange_Interpolation(10))

14.666666666666666


In [84]:
from sympy import symbols, factorial, diff, lambdify, simplify

u = symbols('u')
poly_string = "46 + ((u * 20) / 1!) + ((u * (u - 1) * -5) / 2!) + ((u * (u - 1) * (u - 2) * 2) / 3!) + ((u * (u - 1) * (u - 2) * (u - 3) * -3) / 4!)"
polynomial = Poly(poly_string, u)
polynomial

Poly(-1/8*u**4 + 13/12*u**3 - 39/8*u**2 + 287/12*u + 46, u, domain='QQ')

In [120]:
# Newton's forward, backward and divided difference

import math
import numpy as np

class Interpolation:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.forward_diff_row = []
        self.backward_diff_row = []
        self.divided_diff = []
        self.gauss_forward_diff = []
        self.gauss_backward_diff = []
        self.difference_table()
        self.u = symbols('u')   # for polynomial in u
        self.var = symbols('x') # real variable for polynomial

    def difference_table(self):
        # self.forward_diff_row = []
        # self.backward_diff_row = []
        curr_col = self.y.copy()
        curr_len = len(curr_col)
        
        while(curr_len > 1):
            for i in range (0, curr_len - 1):
                sub = curr_col[i + 1] - curr_col[i]
                if i == 0:
                    self.forward_diff_row.append(sub)
                if i == (curr_len - 2):
                    self.backward_diff_row.append(sub)
                curr_col.append(sub)
            curr_col = curr_col[curr_len:]
            curr_len = len(curr_col)  

    def calculating_u(self, u, n, is_forward):
        temp = 1
        if is_forward:
            for i in range(0, n):
                temp *= (u - i)
        else:
            for i in range(0, n):
                temp *= (u + i)
        return temp

    def calculating_polynomial_u(self, n, is_forward):
        temp = ""
        if is_forward:
            for i in range(0, n):
                if i == 0:
                    temp = "u"
                else:
                    temp += "(u - " + str(i) + ")"
        else:
            for i in range(0, n):
                temp += "(u + " + str(i) + ")"
        return temp

    def newtons_forward_difference(self, x_point):
        h = self.x[1] - self.x[0]
        u = (x_point - self.x[0]) / h
        # self.difference_table()
        n = len(self.forward_diff_row)
        yx = self.y[0]
        for i in range(1, n + 1):  # +1 ensures the last difference (delta y0) is included
            yx = yx + (self.calculating_u(u, i, True) * self.forward_diff_row[i-1] / math.factorial(i))
        return yx

    def newtons_forward_difference_polynomial(self):
        h = self.x[1] - self.x[0]
        u = (self.var - self.x[0]) / h
        poly_expr = self.y[0]
        for i in range(1, len(self.forward_diff_row) + 1):
            term = 1
            for j in range(i):
                term *= (u - j)
            poly_expr += (term * self.forward_diff_row[i - 1]) / factorial(i)
        return simplify(poly_expr.expand())

    def derivative_at_point(self, method="divided", x_val=0):
        if method == "forward":
            poly = self.newtons_forward_difference_polynomial()
        elif method == "backward":
            poly = self.newtons_backward_difference_polynomial()
        else:
            poly = self.newtons_divided_difference_polynomial()

        derivative = diff(poly, self.var)
        f_prime = lambdify(self.var, derivative, "numpy")
        return float(f_prime(x_val)), poly, derivative

    def newtons_backward_difference_polynomial(self):
        h = self.x[1] - self.x[0]
        u = (self.var - self.x[-1]) / h
        poly_expr = self.y[-1]
        for i in range(1, len(self.backward_diff_row) + 1):
            term = 1
            for j in range(i):
                term *= (u + j)
            poly_expr += (term * self.backward_diff_row[i - 1]) / factorial(i)
        return simplify(poly_expr.expand())

    def newtons_backward_difference(self, x_point):
        h = self.x[1] - self.x[0]
        u = (x_point - self.x[len(self.x) - 1]) / h
        # self.difference_table()
        n = len(self.backward_diff_row)
        yx = self.y[len(self.y) - 1]
        for i in range(1, n + 1):  
            yx += (self.calculating_u(u, i, False) * self.backward_diff_row[i-1] / math.factorial(i))
        return yx

    def divided_difference_table(self):
        n = len(self.x)
        table = [[0 for _ in range(n)] for __ in range(n)]
        for i in range(n):
            table[i][0] = self.y[i]
        for j in range(1, n):
            for i in range(n - j):
                table[i][j] = (table[i + 1][j - 1] - table[i][j - 1]) / (self.x[i + j] - self.x[i])
        self.divided_diff = [table[0][j] for j in range(n)]
        return table

    def newtons_divided_difference_polynomial(self):
        if not self.divided_diff:
            self.divided_difference_table()
        poly_expr = self.divided_diff[0]
        term = 1
        for i in range(1, len(self.x)):
            term *= (self.var - self.x[i - 1])
            poly_expr += self.divided_diff[i] * term
        return simplify(poly_expr.expand())

In [123]:
x = [0.0, 0.1, 0.2, 0.3, 0.4]
y = [1.0, 0.9975, 0.99, 0.9776, 0.8604]
# x = [2,4,9,10]
# y = [4,56,711,980]

interp = Interpolation(x, y)

# Get polynomial (divided difference, works for unequal intervals too)
poly = interp.newtons_divided_difference_polynomial()
print("Polynomial:", poly)

# Derivative and evaluation at x=2.5
dy_val, poly_expr, derivative_expr = interp.derivative_at_point(method="divided", x_val=0)
print("Polynomial expanded:", poly_expr)
print("Derivative:", derivative_expr)
print("dy/dx at x:", dy_val)

Polynomial: -41.6666666666668*x**4 + 25.0166666666668*x**3 - 4.83833333333336*x**2 + 0.250333333333336*x + 1.0
Polynomial expanded: -41.6666666666668*x**4 + 25.0166666666668*x**3 - 4.83833333333336*x**2 + 0.250333333333336*x + 1.0
Derivative: -166.666666666667*x**3 + 75.0500000000003*x**2 - 9.67666666666672*x + 0.250333333333336
dy/dx at x: 0.250333333333336
